In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
sns.set_style("ticks")
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'
import shutil
from os import listdir
from os.path import join, isfile
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
! pip install XlsxWriter
import xlsxwriter

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 3.3 MB/s eta 0:00:00


 Very important!!! delete all the already created dashboards in the "finished dashboards" folder


In [ ]:
rev_newer=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/revenue_only.csv")
rev_newer["Date Received"] = pd.to_datetime(rev_newer["Date Received"])
rev_newer =rev_newer.set_index("Date Received")
rev_newer["Year"]=rev_newer["Year"].astype(str)
rev_newer['Quarter']=rev_newer['Quarter'].astype(str)
rev_newer["Quarter_to_match"]=rev_newer["Year"]+"Q"+rev_newer['Quarter']

In [ ]:
# read forecast
def forecast_data(country):
    #forecast test.xlsx
    #total=pd.read_excel("forecast test.xlsx", sheet_name=country) 5qrf copy for python final.xlsx
    total=pd.read_excel("/content/drive/MyDrive/Colab Notebooks/5qrf copy for python final.xlsx", sheet_name=country)

    total=total[total["Unnamed: 0"].str.contains("volume forecast")==True ]
    test_header_col=total["Unnamed: 0"]
    total= total.filter(regex=r'^(?!Unnamed).*')
    total.fillna(0,inplace=True)
    start_date = '2022-01-01'
    end_date = '2024-08-30'
    quarters = pd.period_range(start=start_date, end=end_date, freq='Q')
    quarters_list = [str(q) for q in quarters]
    fin_df=pd.DataFrame()
    fin_df["header"]=test_header_col
    quarter=0
    for col in total.columns:
        target_col=quarters_list[quarter]
        quarter+=1
        fin_df[target_col]=total[col]
    return fin_df

In [ ]:
def new_forecast_data(country,filepath):
    path="/content/drive/MyDrive/Colab Notebooks/"
    filepath=path+filepath
    total=pd.read_excel(filepath, sheet_name=country)

    total=total[total["Unnamed: 0"].str.contains("volume forecast")==True ]
    test_header_col=total["Unnamed: 0"]
    total= total.filter(regex=r'^(?!Unnamed).*')
    total.fillna(0,inplace=True)
    start_date = '2022-01-01'
    end_date = '2024-08-30'
    quarters = pd.period_range(start=start_date, end=end_date, freq='Q')
    quarters_list = [str(q) for q in quarters]
    fin_df=pd.DataFrame()
    fin_df["header"]=test_header_col
    quarter=0
    for col in total.columns:
        target_col=quarters_list[quarter]
        quarter+=1
        fin_df[target_col]=total[col]
    return fin_df


In [22]:
def get_actuals_month(df, country,start,stop):
  if country=="no filter":
    pass
  elif country=="South America":
    df=df[df["adv_Sub_region"]=="South America"]
  elif country=="Rest":
    filter_list=['GB', 'ES', 'TR'] #['GB', 'ES', 'TR','IT','FR']
    df=df[(df["adv_Sub_region"]=="Europe")&(df["Reporting Country  ↑"].isin(filter_list)==False)]
  else:
    df=df[df["Reporting Country"]==country]

  df=df.loc[start:stop]
  unique_quarters=df["Quarter_to_match"].unique()
  bigger_df=pd.DataFrame({"header":['G360 CDx/74 genes sample volume forecast',
       'G360  83 genes sample volume forecast',
       'Reveal sample volume forecast', 'Shield volume forecast']})
  for quarter in unique_quarters:
    short_df=df[df["Quarter_to_match"]==quarter]
    short_dict=short_df["new LIMS Project ID"].value_counts().to_dict()
    keys = list(short_dict.keys())
    if 'Guardant360 CDx' in keys:
      pass
    else:
      short_dict['Guardant360 CDx']=0
    if 'G360/2.12' in keys:
      pass
    else:
      short_dict['G360/2.12']=0
    if 'Reveal' in keys:
      pass
    else:
      short_dict['Reveal']=0
    single_col=[short_dict['Guardant360 CDx'], short_dict['G360/2.12'],short_dict['Reveal'],0] # for reveal
    bigger_df[quarter]= single_col
  return bigger_df
#get_actuals_month(rev_newer, "SPAIN","2022-08-01","2022-08-31")

In [ ]:
def get_actuals_shape(df, country):
  if country=="no filter":
    pass
  elif country=="South America":
    df=df[df["adv_Sub_region"]=="South America"]
  elif country=="Rest":
    filter_list=['GB', 'ES', 'TR'] #['GB', 'ES', 'TR','IT','FR']
    df=df[(df["adv_Sub_region"]=="Europe")&(df["Reporting Country  ↑"].isin(filter_list)==False)]
  else:
    df=df[df["Reporting Country"]==country]
  unique_quarters=df["Quarter_to_match"].unique()
  bigger_df=pd.DataFrame({"header":['G360 CDx/74 genes sample volume forecast',
       'G360  83 genes sample volume forecast',
       'Reveal sample volume forecast', 'Shield volume forecast']})
  for quarter in unique_quarters:
    short_df=df[df["Quarter_to_match"]==quarter]
    short_dict=short_df["new LIMS Project ID"].value_counts().to_dict()
    keys = list(short_dict.keys())
    if 'Guardant360 CDx' in keys:
      pass
    else:
      short_dict['Guardant360 CDx']=0
    if 'G360/2.12' in keys:
      pass
    else:
      short_dict['G360/2.12']=0
    if 'Reveal' in keys:
      pass
    else:
      short_dict['Reveal']=0
    single_col=[short_dict['Guardant360 CDx'], short_dict['G360/2.12'],short_dict['Reveal'],0] # for reveal
    bigger_df[quarter]= single_col
  return bigger_df
#get_actuals_shape(rev_newer, "Rest")

# functions to compare the actuals with the forecast on a quarterly basis

In [ ]:
# Quarter
# entire EUAA region, over/ under devided by products
def euaa_comp_actuals(rev_newer,quarter):
  actuals=get_actuals_shape(rev_newer, "no filter")
  actuals.set_index('header', inplace=True)
  forecast_df=forecast_data("Total region")
  forecast_df.set_index('header', inplace=True)
  result=actuals-forecast_df
  result=result[quarter]
  return result
daf=euaa_comp_actuals(rev_newer,"2023Q2")
daf

header
G360 CDx/74 genes sample volume forecast    -13
G360  83 genes sample volume forecast        -4
Reveal sample volume forecast               -11
Shield volume forecast                       -5
Name: 2023Q2, dtype: object

In [ ]:
# Quarter
# over/ under devided by the countries
def country_comparison_actuals(comp_quarter):
  country_value_dict={}
  country_looper_list=["Canada","Spain","Turkey","Uk final","Latam", "Rest_EUAA"]
  actual_country_dict={"Canada":"CANADA","Spain":"SPAIN","Turkey":"TURKEY","Uk final":"UNITED KINGDOM","Latam":"South America", "Rest_EUAA":"Rest"}
  for country in country_looper_list:
    actual_country=actual_country_dict[country]
    new_forecast=forecast_data(country)
    new_forecast=new_forecast[new_forecast["header"]!="Shield volume forecast" ]
    new_forecast.set_index('header', inplace=True)
    actuals=get_actuals_shape(rev_newer,actual_country)
    actuals=actuals[actuals["header"]!="Shield volume forecast"]
    actuals.set_index('header', inplace=True)

    difference_old_new = actuals - new_forecast
    summed_df = difference_old_new.sum().to_dict()
    country_value_dict[country]=summed_df[comp_quarter]
  df=pd.DataFrame(list(country_value_dict.items()), columns=['Country', 'Gap'])
  return df
dfe=country_comparison_actuals("2023Q2")
dfe


,Country,Gap
0,Canada,-3.0
1,Spain,-24.0
2,Turkey,12.0
3,Uk final,6.0
4,Latam,-22.0
5,Rest_EUAA,3.0


In [34]:
# Quarter
#single country over/ under
def single_country_comparison(rev_newer,quarter,country):
  actual_country_dict={"Canada":"CANADA","Spain":"SPAIN","Turkey":"TURKEY","Uk final":"UNITED KINGDOM","Latam":"South America", "Rest_EUAA":"Rest"}
  actual_country=actual_country_dict[country]
  actuals=get_actuals_shape(rev_newer, actual_country)
  actuals.set_index('header', inplace=True)
  forecast_df=forecast_data(country)
  forecast_df.set_index('header', inplace=True)
  result=actuals-forecast_df
  result=result[quarter]
  return result

single_country_comparison(rev_newer,"2023Q2","Spain")

header
G360 CDx/74 genes sample volume forecast   -25
G360  83 genes sample volume forecast        1
Reveal sample volume forecast                0
Shield volume forecast                       0
Name: 2023Q2, dtype: int64

# all functions below are for a monthly basis

In [35]:
# Monthly
# entire EUAA region, over/ under devided by products
def euaa_comp_actuals(rev_newer,quarter,start,stop):
  actuals=get_actuals_month(rev_newer,"no filter",start,stop)
  actuals.set_index('header', inplace=True)
  forecast_df=forecast_data("Total region")
  forecast_df[quarter]=forecast_df[quarter]/3
  forecast_df.set_index('header', inplace=True)
  result=actuals-forecast_df
  result=result[quarter]
  return result
daf=euaa_comp_actuals(rev_newer,"2023Q3","2023-08-01","2023-08-31")
daf

header
G360 CDx/74 genes sample volume forecast   -63.0
G360  83 genes sample volume forecast        5.0
Reveal sample volume forecast              -9.33
Shield volume forecast                     -8.33
Name: 2023Q3, dtype: object

In [44]:
# Monthly
# over/ under devided by the countries
def country_comparison_actuals(comp_quarter,start,stop):
  country_value_dict={}
  country_looper_list=["Canada","Spain","Turkey","Uk final","Latam", "Rest_EUAA"]
  actual_country_dict={"Canada":"CANADA","Spain":"SPAIN","Turkey":"TURKEY","Uk final":"UNITED KINGDOM","Latam":"South America", "Rest_EUAA":"Rest"}
  for country in country_looper_list:
    actual_country=actual_country_dict[country]
    new_forecast=forecast_data(country)
    new_forecast=new_forecast[new_forecast["header"]!="Shield volume forecast" ]
    new_forecast[comp_quarter]=new_forecast[comp_quarter]/3
    new_forecast.set_index('header', inplace=True)
    actuals=get_actuals_month(rev_newer,actual_country,start,stop)
    actuals=actuals[actuals["header"]!="Shield volume forecast"]
    actuals.set_index('header', inplace=True)

    difference_old_new = actuals - new_forecast
    summed_df = difference_old_new.sum().to_dict()
    country_value_dict[country]=summed_df[comp_quarter]
  df=pd.DataFrame(list(country_value_dict.items()), columns=['Country', 'Gap'])
  return df
country_comparison_actuals("2023Q3","2023-08-01","2023-08-31")



,Country,Gap
0,Canada,-2.33
1,Spain,-6.33
2,Turkey,-6.00
3,Uk final,-30.67
4,Latam,-19.00
5,Rest_EUAA,-3.00


In [43]:
# monthly
#single country over/ under
def single_country_comparison(rev_newer,quarter,country,start,stop):
  actual_country_dict={"Canada":"CANADA","Spain":"SPAIN","Turkey":"TURKEY","Uk final":"UNITED KINGDOM","Latam":"South America", "Rest_EUAA":"Rest"}
  actual_country=actual_country_dict[country]
  actuals=get_actuals_month(rev_newer,actual_country,start,stop)
  actuals.set_index('header', inplace=True)
  forecast_df=forecast_data(country)
  forecast_df.set_index('header', inplace=True)
  forecast_df[quarter]=forecast_df[quarter]/3
  result=actuals-forecast_df
  result=result[quarter]
  return result

single_country_comparison(rev_newer,"2023Q3","Uk final","2023-08-01","2023-08-31")

header
G360 CDx/74 genes sample volume forecast   -24.0
G360  83 genes sample volume forecast       -2.0
Reveal sample volume forecast              -4.67
Shield volume forecast                       0.0
Name: 2023Q3, dtype: object